In [33]:
import pennylane as qml
from pennylane import numpy as np
import tensorflow as tf

In [34]:
# Define quantum device and circuit
n_qubits = 2
dev = qml.device("default.qubit", wires=n_qubits)

In [35]:
@qml.qnode(dev, interface="tf")
def quantum_circuit(inputs, weights):
    # Use the shape information for tensor compatibility
    for i in range(inputs.shape[-1]):  # Loop through input features
        qml.RY(inputs[i], wires=i)
    qml.CNOT(wires=[0, 1])
    qml.Rot(weights[0], weights[1], weights[2], wires=0)
    return qml.expval(qml.PauliZ(0))

In [36]:
# TensorFlow model integration
inputs = tf.keras.Input(shape=(n_qubits,))  # Input layer with shape (n_qubits,)
inputs

<KerasTensor shape=(None, 2), dtype=float32, sparse=False, name=keras_tensor_11>

In [37]:
weights = tf.Variable(tf.random.uniform((3,)))  # Trainable weights for the quantum circuit
weights

<tf.Variable 'Variable:0' shape=(3,) dtype=float32, numpy=array([0.44531763, 0.4495566 , 0.29686534], dtype=float32)>

In [38]:
# Lambda layer with specified output shape
outputs = tf.keras.layers.Lambda(
    lambda x: quantum_circuit(x, weights),  # Quantum circuit applied to inputs
    output_shape=(1,)  # Specify output shape explicitly
)(inputs)
outputs

<KerasTensor shape=(None, 1), dtype=float32, sparse=False, name=keras_tensor_12>

In [39]:
# Build and compile the model
model = tf.keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer="adam", loss="mse")

In [40]:
# Example data
X = np.random.rand(10, n_qubits)  # 10 samples with 2 features each
y = np.random.rand(10, 1)  # Random target values

In [41]:
# Train the model
model.fit(X, y, epochs=10, batch_size=2)

Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0820  
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.1514 
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0910
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.1665 
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.1409 
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.1622 
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.1488 
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.1290 
Epoch 9/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.1421
Epoch 10/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.1990 
